### Modele de classification des messages haineux et non haineux dans le contexte Camerounais

### TCHIAZE FOUOSSO ROMERO
### NDONKOU FRANCK
### ENGOULOU GAETAN

In [ ]:

# --- Étape 0 : Installation des Dépendances Compatibles et Redémarrage ---
# Exécutez cette cellule UNE SEULE FOIS au début de votre session.
# Elle va forcer le redémarrage du noyau, ce qui est NORMAL.
# Après le redémarrage, ne la ré-exécutez pas et passez à la cellule suivante.
import os

print("Installation des versions de bibliothèques compatibles pour un entraînement stable...")
# On utilise un ensemble de versions connues pour bien fonctionner ensemble.
# `transformers==4.41.2` est une version stable qui fonctionne bien avec `peft` récent.
!pip install transformers==4.41.2 datasets==2.19.1 sentencepiece==0.2.0 accelerate==0.30.1 peft==0.10.0 scikit-learn seaborn -q
print("Installation terminée.")

print("\nREDÉMARRAGE DU KERNEL pour appliquer les changements et nettoyer l'état du GPU...")
print("C'est normal. Ne ré-exécutez pas cette cellule après le redémarrage.")
os.kill(os.getpid(), 9)

Model page: https://huggingface.co/Poulpidot/distilcamenbert-french-hate-speech

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Poulpidot/distilcamenbert-french-hate-speech)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

# Projet de Détection de Discours Haineux : Fine-Tuning Avancé

**Objectif :** Adapter le modèle `Poulpidot/distilcamenbert-french-hate-speech` à un corpus de textes spécifiques (contenant du vocabulaire camerounais) pour améliorer la performance de classification.

**Démarche en 2 phases :**
1.  **Phase 1 : Adaptation au Domaine (MLM)**
    - Entraînement d'un nouveau tokenizer sur notre corpus.
    - Fine-tuning du modèle pré-entraîné via le *Masked Language Modeling* (MLM) pour qu'il apprenne notre vocabulaire.
2.  **Phase 2 : Fine-tuning pour la Classification**
    - Utilisation du modèle adapté en Phase 1 comme base.
    - Fine-tuning sur la tâche de classification binaire (hateful / not_hateful).

**Résultats attendus :** Un modèle de classification robuste, des visualisations de performance (perte, précision) et une matrice de confusion pour l'évaluation finale.

In [ ]:
import os
import torch
import logging

# Configurer le logging pour plus de verbosité
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Étape 1 : Installer les dépendances nécessaires
try:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
    logger.info("Transformers déjà installé.")
except ImportError:
    logger.info("Installation de transformers et dépendances...")
    try:
        !pip install transformers==4.45.2 torch sentencepiece
        from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
        logger.info("Transformers installé avec succès.")
    except Exception as e:
        logger.error(f"Erreur lors de l'installation de transformers : {e}")
        raise

# Définir le nom du modèle
model_name = "Poulpidot/distilcamenbert-french-hate-speech"

# Définir le dossier de sauvegarde (dans /kaggle/working/ pour Kaggle)
output_dir = "/kaggle/working/distilcamenbert_french_hate_speech"
os.makedirs(output_dir, exist_ok=True)
logger.info(f"Dossier de sauvegarde créé ou existant : {output_dir}")

# Étape 2 : Vérifier si le modèle existe déjà localement
required_files = ['config.json', 'model.safetensors', 'sentencepiece.bpe.model', 'tokenizer.json', 'tokenizer_config.json', 'special_tokens_map.json']
model_exists_locally = all(os.path.exists(os.path.join(output_dir, f)) for f in required_files)

if not model_exists_locally:
    try:
        # Télécharger et sauvegarder le tokenizer
        logger.info(f"Téléchargement du tokenizer depuis {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.save_pretrained(output_dir)
        logger.info("Tokenizer sauvegardé.")

        # Télécharger et sauvegarder le modèle
        logger.info(f"Téléchargement du modèle depuis {model_name}...")
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        model.save_pretrained(output_dir)
        logger.info("Modèle sauvegardé.")
    except Exception as e:
        logger.error(f"Erreur lors du téléchargement ou de la sauvegarde : {e}")
        raise
else:
    logger.info(f"Modèle déjà présent dans {output_dir}. Chargement local.")

# Vérifier que les fichiers sont bien sauvegardés
print("Modèle et tokenizer sauvegardés dans :", output_dir)
print("Fichiers présents :", os.listdir(output_dir))

# Étape 3 : Tester le modèle localement avec une phrase
try:
    # Charger la pipeline depuis le dossier local
    logger.info(f"Chargement de la pipeline depuis {output_dir}...")
    pipe = pipeline("text-classification", model=output_dir, tokenizer=output_dir, device=0 if torch.cuda.is_available() else -1)

    # Phrase à tester
    phrase = "Cette personne est vraiment méchante et inutile."

    # Faire une prédiction
    result = pipe(phrase)

    # Afficher le résultat
    print(f"Phrase : {phrase}")
    print(f"Prédiction : {result}")
except Exception as e:
    logger.error(f"Erreur lors de l'utilisation de la pipeline : {e}")
    raise

# Étape 4 : Tester avec le tokenizer et le modèle directement
try:
    # Charger le tokenizer et le modèle localement
    tokenizer = AutoTokenizer.from_pretrained(output_dir)
    model = AutoModelForSequenceClassification.from_pretrained(output_dir)

    # Tokeniser la phrase
    inputs = tokenizer(phrase, return_tensors="pt", padding="max_length", truncation=True, max_length=100)

    # Évaluer le modèle
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        label_idx = torch.argmax(probs, dim=-1).item()
        labels = {0: "not_hateful", 1: "hateful"}  # Labels confirmés pour le modèle

    print(f"Prédiction détaillée : {labels[label_idx]} (probabilité : {probs[0][label_idx]:.4f})")
except Exception as e:
    logger.error(f"Erreur lors du test manuel : {e}")
    raise

In [ ]:
"""import os
import shutil

# Chemin du dossier à compresser
folder_path = "/kaggle/working/distilcamenbert_french_hate_speech"

# Nom du fichier ZIP à créer
zip_file_name = "distilcamenbert_french_hate_speech.zip"

# Compresser le dossier en ZIP
shutil.make_archive(zip_file_name[:-4], 'zip', folder_path)

# Vérifier que le fichier ZIP a été créé
print("Fichier ZIP créé :", os.path.join("/kaggle/working", zip_file_name))"""

In [ ]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import spacy

# --- GESTION DE L'INSTALLATION (si nécessaire) ---
# Si le modèle n'est pas trouvé, le code suivant tentera de l'installer.
try:
    nlp = spacy.load("fr_core_news_sm")
    print("Modèle SpaCy 'fr_core_news_sm' chargé avec succès.")
except OSError:
    print("Modèle SpaCy 'fr_core_news_sm' non trouvé. Tentative de téléchargement...")
    try:
        spacy.cli.download("fr_core_news_sm")
        nlp = spacy.load("fr_core_news_sm")
        print("Modèle téléchargé et chargé avec succès.")
    except Exception as e:
        print(f"Échec du téléchargement automatique du modèle SpaCy : {e}")
        exit()

# Télécharger les stop words pour le français
nltk.download('stopwords', quiet=True)
french_stopwords = set(stopwords.words('french'))


# --- CHEMINS DES FICHIERS ---
input_file = "/kaggle/input/datacamer/dataMessages_filtered"
output_file = "/kaggle/working/FirstDataSet_Whatsapp_Youtube_processed_final.csv"

# Vérifier si le fichier d'entrée existe
if not os.path.exists(input_file):
    print(f"Erreur : Le fichier {input_file} n'a pas été trouvé.")
    # Si le chemin est incorrect, cette ligne aide à trouver le bon.
    # print("Vérifiez le contenu de /kaggle/input/ :", os.listdir("/kaggle/input/"))
    exit()


# --- NOUVELLES FONCTIONS DE PRÉTRAITEMENT ---
# Ces fonctions sont ajoutées sans modifier les vôtres.

def normalize_repeated_chars(text):
    """Normalise les caractères répétés plus de deux fois. Ex: 'troooop' -> 'troop'."""
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

def lemmatize_text_spacy(text):
    """
    Effectue la lemmatisation en utilisant SpaCy.
    C'est une étape plus avancée que la simple suppression de stopwords.
    Ex: "les voitures roulaient vite" -> "le voiture rouler vite"
    """
    # On désactive le parser et la reconnaissance d'entités pour la vitesse.
    doc = nlp(text, disable=['parser', 'ner'])
    return " ".join([token.lemma_ for token in doc])


# --- VOS FONCTIONS DE PRÉTRAITEMENT (INCHANGÉES) ---

def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    return re.sub(entity_regex, "", raw_text)

def change_user(raw_text):
    regex = r"@([^ ]+)"
    return re.sub(regex, "user", raw_text)

def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    return re.sub(url_regex, '', raw_text)

def remove_noise_symbols(raw_text):
    return raw_text.replace('"', '').replace("'", '').replace("!", '').replace("`", '').replace("..", '')

def remove_punctuation(text):
    return "".join(c for c in text if c not in string.punctuation)

def remove_stopwords(text):
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in french_stopwords])

def remove_names(text):
    try:
        doc = nlp(text)
        return ' '.join([token.text for token in doc if token.ent_type_ not in ('PER', 'LOC', 'ORG')])
    except Exception as e:
        print(f"Erreur dans remove_names : {e}")
        return text


# --- PIPELINE DE PRÉTRAITEMENT INTÉGRANT LES NOUVELLES ÉTAPES ---

def preprocess_text(text):
    """
    Applique toutes les étapes de prétraitement, anciennes et nouvelles,
    dans un ordre logique et optimisé.
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Nettoyage initial (URLs, mentions, entités)
    text = remove_url(text)
    text = change_user(text)
    text = remove_entity(text)
    
    # 2. Normalisation du texte
    text = text.lower()  # Mise en minuscule
    text = normalize_repeated_chars(text) # NOUVELLE ÉTAPE
    
    # 3. Lemmatisation (gère les formes de mots)
    # text = lemmatize_text_spacy(text) # NOUVELLE ÉTAPE (Optionnelle, puissante mais lente)
    # NOTE: La lemmatisation est très puissante mais peut être lente.
    # Pour commencer, je la laisse commentée. Décommentez-la pour un nettoyage plus profond.

    # 4. Suppression du bruit (symboles, ponctuation)
    text = remove_noise_symbols(text)
    text = remove_punctuation(text)
    
    # 5. Suppression des mots non pertinents (stopwords, noms)
    text = remove_stopwords(text)
    text = remove_names(text)
    
    # 6. Nettoyage final des espaces
    text = ' '.join(text.split())
    
    return text


# --- FONCTION DE STATISTIQUES (INCHANGÉE) ---

def compute_statistics(df, text_column, label_column, title_prefix=""):
    print(f"\n=== {title_prefix} Statistiques ===")
    print(f"Nombre total d'exemples : {len(df)}")
    
    class_counts = df[label_column].value_counts()
    print("\nDistribution des classes :"); print(class_counts)
    
    df['word_count'] = df[text_column].apply(lambda x: len(str(x).split()))
    print("\nStatistiques sur la longueur des phrases (en mots) :"); print(df['word_count'].describe())
    
    all_words = ' '.join(df[text_column].astype(str)).split()
    word_freq = Counter(all_words).most_common(10)
    print("\n10 mots les plus fréquents :"); print(word_freq)
    
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 2, 1)
    sns.countplot(x=label_column, data=df); plt.title(f"{title_prefix} Distribution des classes")
    
    plt.subplot(2, 2, 2)
    plt.hist(df['word_count'], bins=20, edgecolor='black'); plt.title(f"{title_prefix} Histogramme de la longueur des phrases")
    
    plt.subplot(2, 2, 3)
    words, freqs = zip(*word_freq)
    sns.barplot(x=list(freqs), y=list(words)); plt.title(f"{title_prefix} 10 mots les plus fréquents")
    
    plt.tight_layout()
    plt.savefig(f"/kaggle/working/statistics_{title_prefix.lower().replace(' ', '_')}.png")
    plt.show()
    
    df.drop(columns=['word_count'], inplace=True, errors='ignore')
    return df


# --- EXÉCUTION DU SCRIPT ---

try:
    df = pd.read_csv(input_file, encoding='utf-8')
except Exception as e:
    print(f"Erreur lors de la lecture du fichier {input_file} : {e}")
    exit()

if not all(col in df.columns for col in ['message', 'vote_final']):
    print("Erreur : Le fichier CSV doit contenir les colonnes 'message' et 'vote_final'.")
    exit()

# Étape 1 : Étude statistique avant prétraitement
df = compute_statistics(df, text_column='message', label_column='vote_final', title_prefix="Avant prétraitement")

# Étape 2 : Prétraitement des phrases
print("\nApplication du prétraitement sur les messages...")
from tqdm.auto import tqdm
tqdm.pandas() # Active la barre de progression pour .apply()
df['message_cleaned'] = df['message'].progress_apply(preprocess_text)
print("Prétraitement terminé.")

# Étape 3 : Étude statistique après prétraitement
df = compute_statistics(df, text_column='message_cleaned', label_column='vote_final', title_prefix="Après prétraitement")

# Étape 4 : Sauvegarder le fichier CSV prétraité
try:
    output_df = df[['message_cleaned', 'vote_final']].rename(columns={'message_cleaned': 'message'})
    output_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nFichier CSV prétraité sauvegardé : {output_file}")
    print("Aperçu des premières lignes :")
    print(output_df.head())
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier {output_file} : {e}")
    exit()

In [ ]:
# Définir les chemins des fichiers
input_file = "/kaggle/input/datacamer/dataMessages_filtered"  # Chemin du fichier d'entrée
output_file = "/kaggle/working/FirstDataSet_Whatsapp_Youtube_processed_final.csv"  # Chemin de sortie

# Charger le fichier CSV
try:
    df = pd.read_csv(input_file, encoding='utf-8')
except Exception as e:
    print(f"Erreur lors de la lecture du fichier {input_file} : {e}")
    exit()

# Vérifier la structure du fichier
if not all(col in df.columns for col in ['message', 'vote_final']):
    print("Erreur : Le fichier CSV doit contenir les colonnes 'message' et 'vote_final'.")
    exit()

# Étape 1 : Étude statistique avant prétraitement
df = compute_statistics(df, text_column='message', label_column='vote_final', title_prefix="Avant prétraitement")

# Étape 2 : Prétraitement des phrases
df['message_cleaned'] = df['message'].apply(preprocess_text)

# Étape 3 : Étude statistique après prétraitement
df = compute_statistics(df, text_column='message_cleaned', label_column='vote_final', title_prefix="Après prétraitement")

# Étape 4 : Sauvegarder le fichier CSV prétraité
try:
    output_df = df[['message_cleaned', 'vote_final']].rename(columns={'message_cleaned': 'message'})
    output_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\nFichier CSV prétraité sauvegardé : {output_file}")
    print("Aperçu des premières lignes :")
    print(output_df.head())
except Exception as e:
    print(f"Erreur lors de la sauvegarde du fichier {output_file} : {e}")
    exit()

In [ ]:
# Cellule 2 : Imports et Configuration (APRÈS REDÉMARRAGE)

import os
import pandas as pd
import torch
import logging
from tqdm.auto import tqdm
import re
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from transformers.trainer_callback import TrainerCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
os.environ["WANDB_DISABLED"] = "true"

# --- Définition des Chemins et Paramètres ---
BASE_MODEL_DIR = "/kaggle/working/distilcamenbert_french_hate_speech"
DATASET_PATH = "/kaggle/working/FirstDataSet_Whatsapp_Youtube_processed_final.csv"
EXTENDED_MODEL_DIR = "/kaggle/working/distilcamembert_extended"
ADAPTED_LM_DIR = "/kaggle/working/distilcamembert_extended_adapted_lm"
FINAL_CLASSIFIER_DIR = "/kaggle/working/final_hate_speech_classifier"
MLM_EPOCHS = 15 # Élevé, car Early Stopping décidera
CLASSIFICATION_EPOCHS = 20 # Élevé, car Early Stopping décidera
BATCH_SIZE = 16
LEARNING_RATE = 5e-6 # Taux d'apprentissage faible et stable

logger.info("Configuration et imports terminés.")

In [ ]:
# --- Étape 1.1 : Extension du Tokenizer et du Modèle ---
logger.info("Phase 1 : Adaptation du modèle au langage spécifique.")

logger.info("Chargement du dataset et du tokenizer original...")
df = pd.read_csv(DATASET_PATH).dropna(subset=['message', 'vote_final'])
df['message'] = df['message'].astype(str)
original_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR)
original_vocab = set(original_tokenizer.get_vocab().keys())

logger.info("Identification des mots absents du vocabulaire original...")
def extract_words(text): return set(re.findall(r"[\w']+", text.lower()))
corpus_words = set()
for text in tqdm(df['message'], desc="Analyse du corpus"): corpus_words.update(extract_words(text))
new_tokens = list(corpus_words - original_vocab)
word_counts = pd.Series(' '.join(df['message'].str.lower()).split()).value_counts()
new_tokens_to_add = [token for token in new_tokens if word_counts.get(token, 0) > 1]

logger.info(f"Ajout de {len(new_tokens_to_add)} nouveaux mots pertinents au tokenizer.")
original_tokenizer.add_tokens(new_tokens_to_add)

logger.info("Chargement et redimensionnement du modèle de base...")
model_for_extension = AutoModelForMaskedLM.from_pretrained(BASE_MODEL_DIR)
model_for_extension.resize_token_embeddings(len(original_tokenizer))

logger.info(f"Sauvegarde du couple modèle/tokenizer étendu dans {EXTENDED_MODEL_DIR}...")
os.makedirs(EXTENDED_MODEL_DIR, exist_ok=True)
model_for_extension.save_pretrained(EXTENDED_MODEL_DIR)
original_tokenizer.save_pretrained(EXTENDED_MODEL_DIR)

In [ ]:
# --- Étape 1.2 : Fine-tuning MLM avec Stabilisation, Métriques et Compatibilité Maximale ---

# Imports nécessaires pour cette cellule
import torch
import math
import pandas as pd
import matplotlib.pyplot as plt
from transformers import (
    TrainerCallback, EarlyStoppingCallback, Trainer, TrainingArguments, 
    __version__ as transformers_version  # Import pour détecter la version
)
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling
from datasets import Dataset
from tqdm.auto import tqdm
from packaging import version # Outil pour comparer les versions

# --- Callbacks Personnalisés ---

class ProgressCallback(TrainerCallback):
    """Callback pour afficher une barre de progression tqdm personnalisée."""
    def __init__(self, task_name="Fine-tuning"):
        self.task_name = task_name
        self.progress_bar = None
    def on_train_begin(self, args, state, control, **kwargs):
        self.progress_bar = tqdm(total=state.max_steps, desc=self.task_name)
    def on_step_end(self, args, state, control, **kwargs):
        self.progress_bar.update(1)
        if state.log_history and 'loss' in state.log_history[-1]:
            loss_value = state.log_history[-1]['loss']
            if loss_value is not None and not torch.isnan(torch.tensor(loss_value)):
                self.progress_bar.set_postfix(loss=f"{loss_value:.4f}")
            else:
                self.progress_bar.set_postfix(loss="NaN")
    def on_train_end(self, args, state, control, **kwargs):
        if self.progress_bar: self.progress_bar.close()

class NanLossStopper(TrainerCallback):
    """Callback pour arrêter l'entraînement si la perte devient NaN."""
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs and (logs['loss'] is None or torch.isnan(torch.tensor(logs['loss']))):
            logger.error("Erreur fatale : Perte NaN détectée. Arrêt de l'entraînement.")
            control.should_training_stop = True

class PerplexityCallback(TrainerCallback):
    """Callback qui calcule et ajoute la perplexité aux logs après chaque évaluation."""
    def on_log(self, args, state, control, logs=None, **kwargs):
        # Cette méthode est appelée chaque fois que des logs sont créés.
        # On vérifie si ce sont des logs d'évaluation (qui contiennent 'eval_loss').
        if logs is not None and "eval_loss" in logs:
            try:
                # Calcul de la perplexité
                perplexity = math.exp(logs["eval_loss"])
                # Ajout de la nouvelle métrique au dictionnaire de logs
                logs["eval_perplexity"] = perplexity
            except OverflowError:
                logs["eval_perplexity"] = float("inf")

# --- Préparation du Modèle et des Données ---
logger.info("Chargement du modèle étendu pour le fine-tuning MLM...")
mlm_model = AutoModelForMaskedLM.from_pretrained(EXTENDED_MODEL_DIR)
mlm_tokenizer = AutoTokenizer.from_pretrained(EXTENDED_MODEL_DIR)

logger.info("Préparation du dataset pour le MLM...")
dataset_mlm = Dataset.from_pandas(df[['message']])
def tokenize_function_mlm(examples):
    return mlm_tokenizer(examples['message'], truncation=True, padding='max_length', max_length=128)
tokenized_dataset_mlm = dataset_mlm.map(tokenize_function_mlm, batched=True, remove_columns=['message'], desc="Tokenizing for MLM")
data_collator_mlm = DataCollatorForLanguageModeling(tokenizer=mlm_tokenizer, mlm=True, mlm_probability=0.15)
train_test_split_mlm = tokenized_dataset_mlm.train_test_split(test_size=0.1, seed=42)

# --- Arguments d'Entraînement avec Gestion de Compatibilité ---
common_args = {
    "output_dir": "/kaggle/working/mlm_results",
    "num_train_epochs": MLM_EPOCHS,
    "learning_rate": 5e-8,
    "per_device_train_batch_size": 8,
    "gradient_accumulation_steps": 2,
    "fp16": torch.cuda.is_available(),
    "max_grad_norm": 1.0,
    "warmup_ratio": 0.1,
    "report_to": "none",
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_perplexity", # La métrique que notre callback va ajouter
    "greater_is_better": False, # Pour la perplexité, plus bas c'est mieux
    "save_total_limit": 2,
    "disable_tqdm": True,
    "seed": 42
}

logger.info(f"Version de Transformers détectée : {transformers_version}")
if version.parse(transformers_version) >= version.parse("4.20.0"):
    logger.info("Utilisation de l'API moderne de TrainingArguments (evaluation_strategy).")
    training_args_mlm = TrainingArguments(**common_args, evaluation_strategy="epoch", logging_strategy="epoch", save_strategy="epoch")
else:
    logger.warning("Utilisation de l'API ancienne de TrainingArguments (eval_steps, logging_steps).")
    steps_per_epoch = len(train_test_split_mlm['train']) // (common_args['per_device_train_batch_size'] * common_args['gradient_accumulation_steps'])
    training_args_mlm = TrainingArguments(**common_args, eval_steps=steps_per_epoch, logging_steps=steps_per_epoch, save_steps=steps_per_epoch)


# --- Création et Lancement du Trainer ---
trainer_mlm = Trainer(
    model=mlm_model,
    args=training_args_mlm,
    train_dataset=train_test_split_mlm['train'],
    eval_dataset=train_test_split_mlm['test'],
    data_collator=data_collator_mlm,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01),
        ProgressCallback(task_name="Fine-tuning (MLM)"),
        NanLossStopper(),
        PerplexityCallback() # Le callback qui va ajouter la métrique
    ]
)

logger.info("Début du fine-tuning MLM (avec calcul de perplexité via Callback)...")
trainer_mlm.train()


# --- SAUVEGARDE ET VISUALISATION DES MÉTRIQUES ---
logger.info(f"Sauvegarde du meilleur modèle adapté par MLM dans : {ADAPTED_LM_DIR}")
trainer_mlm.save_model(ADAPTED_LM_DIR)
mlm_tokenizer.save_pretrained(ADAPTED_LM_DIR)
logger.info("Phase 1 (Adaptation du langage) terminée.")

log_history = trainer_mlm.state.log_history
df_log = pd.DataFrame(log_history)

if not df_log.empty:
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    fig.suptitle("Performances du Fine-Tuning MLM", fontsize=16)
    
    train_logs = df_log[df_log['loss'].notna()].copy()
    eval_logs = df_log[df_log['eval_loss'].notna()].copy()

    if not train_logs.empty and not eval_logs.empty:
        axes[0].plot(train_logs['epoch'], train_logs['loss'], marker='o', label="Training Loss")
        axes[0].plot(eval_logs['epoch'], eval_logs['eval_loss'], marker='o', label="Validation Loss")
        axes[0].set_title("Perte (Loss)"); axes[0].set_xlabel("Époque"); axes[0].set_ylabel("Perte"); axes[0].grid(True); axes[0].legend()
    else:
        axes[0].set_title("Perte (Loss) - Données insuffisantes")

    if 'eval_perplexity' in eval_logs.columns:
        axes[1].plot(eval_logs['epoch'], eval_logs['eval_perplexity'], marker='o', color='green', label="Validation Perplexity")
        axes[1].set_title("Perplexité sur l'ensemble de validation")
        axes[1].set_xlabel("Époque"); axes[1].set_ylabel("Perplexité (plus bas = meilleur)"); axes[1].grid(True); axes[1].legend()
    else:
        axes[1].set_title("Perplexité - Données non trouvées")
        axes[1].text(0.5, 0.5, "'eval_perplexity' non trouvé dans les logs.", ha='center', va='center')
        logger.warning("La métrique 'eval_perplexity' n'a pas été trouvée, le graphique ne sera pas généré.")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

    best_metrics = getattr(trainer_mlm.state, 'best_metric', None)
    if best_metrics:
        logger.info(f"Meilleure performance MLM atteinte : Perplexité = {best_metrics:.4f}")
else:
    logger.warning("Aucun log d'entraînement n'a été généré.")

In [ ]:
# --- Étape 1.3 : Visualisation des Résultats MLM ---
logger.info("Génération du graphique de perte pour la phase MLM...")
log_history_mlm = trainer_mlm.state.log_history
df_log_mlm = pd.DataFrame(log_history_mlm)

train_loss_df = df_log_mlm[df_log_mlm['loss'].notna()]
eval_loss_df = df_log_mlm[df_log_mlm['eval_loss'].notna()]

if not train_loss_df.empty and not eval_loss_df.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(train_loss_df['epoch'], train_loss_df['loss'], marker='o', linestyle='-', label="Training Loss")
    plt.plot(eval_loss_df['epoch'], eval_loss_df['eval_loss'], marker='o', linestyle='-', label="Validation Loss")
    plt.title("Perte durant l'entraînement MLM (Phase 1)")
    plt.xlabel("Époque")
    plt.ylabel("Perte (Loss)")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    logger.warning("Impossible de générer le graphique de perte MLM. Données de log insuffisantes.")

In [ ]:
# --- Étape 1.3 (Révisée) : Clustering Forcé à k=2 et Évaluation avec Étiquettes Réelles ---

# Imports nécessaires pour cette cellule
import numpy as np
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

logger.info("Début de l'évaluation des embeddings par clustering K-Means (k=2) et comparaison avec les étiquettes réelles.")

# --- 1. Génération des Embeddings 768D pour l'ENSEMBLE du Dataset ---
logger.info("Chargement du modèle et du tokenizer adaptés...")
# Assurez-vous que ADAPTED_LM_DIR et df sont définis dans les cellules précédentes
model = AutoModel.from_pretrained(ADAPTED_LM_DIR)
tokenizer = AutoTokenizer.from_pretrained(ADAPTED_LM_DIR)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

sentences = df['message'].tolist()
# Récupérer les étiquettes réelles pour l'évaluation finale
true_labels = df['vote_final'].values 

batch_size = 32
all_embeddings = []

logger.info(f"Génération des embeddings pour {len(sentences)} phrases...")
for i in tqdm(range(0, len(sentences), batch_size), desc="Generating Embeddings (768D)"):
    batch_sentences = sentences[i:i+batch_size]
    inputs = tokenizer(batch_sentences, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
        sum_embeddings = torch.sum(outputs.last_hidden_state * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        batch_embeddings = (sum_embeddings / sum_mask).cpu().numpy()
        all_embeddings.append(batch_embeddings)

embeddings_full_768d = np.concatenate(all_embeddings, axis=0)
logger.info("Normalisation des embeddings 768D...")
scaled_embeddings_full_768d = StandardScaler().fit_transform(embeddings_full_768d)


# --- 2. Clustering avec Mini-Batch K-Means (k=2) sur les Données 768D ---
n_clusters = 2
logger.info(f"Application de l'algorithme Mini-Batch K-Means pour trouver exactement {n_clusters} clusters...")
kmeans = MiniBatchKMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init='auto',
    batch_size=256
)
predicted_clusters = kmeans.fit_predict(scaled_embeddings_full_768d)


# --- 3. Réduction de Dimensionnalité avec t-SNE (pour la visualisation) ---
logger.info("Réduction de la dimensionnalité de 768D à 2D avec t-SNE (peut prendre du temps)...")
tsne = TSNE(
    n_components=2, 
    perplexity=30, 
    metric='cosine', 
    random_state=42,
    n_jobs=-1
)
embeddings_2d_for_viz = tsne.fit_transform(embeddings_full_768d)


# --- 4. Évaluation du Clustering en Comparaison avec les Étiquettes Réelles ---
print("\n" + "="*60)
print("--- RÉSULTATS DU CLUSTERING K-MEANS (k=2) vs ÉTIQUETTES RÉELLES ---")

# a) Métrique interne (qualité des clusters formés)
silhouette_avg = silhouette_score(scaled_embeddings_full_768d, predicted_clusters)
print(f"Score de Silhouette (qualité intrinsèque des clusters) : {silhouette_avg:.4f}")

# b) Métriques externes (comparaison avec les étiquettes réelles)
ari_score = adjusted_rand_score(true_labels, predicted_clusters)
nmi_score = normalized_mutual_info_score(true_labels, predicted_clusters)
print(f"Adjusted Rand Score (ARI) : {ari_score:.4f}")
print(f"Normalized Mutual Information (NMI) : {nmi_score:.4f}")
print("\n(Pour ARI et NMI, 1.0 = correspondance parfaite entre clusters et étiquettes, 0.0 = aucune correspondance)")
print("="*60 + "\n")


# --- 5. Visualisation Comparative ---
logger.info("Génération de la visualisation comparative des clusters...")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10))
fig.suptitle("Comparaison entre Étiquettes Réelles et Clusters K-Means (k=2)", fontsize=20)

# Graphique 1 : Coloration par ÉTIQUETTES RÉELLES
sns.scatterplot(
    x=embeddings_2d_for_viz[:, 0], 
    y=embeddings_2d_for_viz[:, 1],
    hue=true_labels, # <-- Utilise les vraies étiquettes
    palette="viridis", 
    s=10, alpha=0.7, ax=ax1, legend='full'
)
ax1.set_title("1. Visualisation selon les Étiquettes Réelles", fontsize=16)
ax1.set_xlabel("Composante t-SNE 1")
ax1.set_ylabel("Composante t-SNE 2")
ax1.grid(True, linestyle='--', alpha=0.6)

# Graphique 2 : Coloration par CLUSTERS PRÉDITS
sns.scatterplot(
    x=embeddings_2d_for_viz[:, 0], 
    y=embeddings_2d_for_viz[:, 1],
    hue=predicted_clusters, # <-- Utilise les 2 clusters trouvés par K-Means
    palette="plasma", 
    s=10, alpha=0.7, ax=ax2, legend='full'
)
ax2.set_title("2. Visualisation selon les 2 Clusters Prédits", fontsize=16)
ax2.set_xlabel("Composante t-SNE 1")
ax2.set_ylabel("Composante t-SNE 2")
ax2.grid(True, linestyle='--', alpha=0.6)

plt.show()

## Phase 2 : Fine-Tuning pour la Classification

Maintenant que notre modèle comprend notre vocabulaire spécifique, nous allons l'entraîner à la tâche finale : classifier les messages comme "hateful" ou "not_hateful".

### Étape 2.1 : Préparation des données

Nous devons préparer les données en associant chaque message à son étiquette numérique, puis en les divisant en ensembles d'entraînement et de test.

In [ ]:
# --- ÉTAPE DE NETTOYAGE : Libérer l'espace disque ---
import shutil
import os
import logging

logger = logging.getLogger(__name__)

mlm_results_dir = "/kaggle/working/mlm_results"

if os.path.exists(mlm_results_dir):
    logger.info(f"Nettoyage de l'espace disque en supprimant les checkpoints MLM intermédiaires de '{mlm_results_dir}'...")
    try:
        shutil.rmtree(mlm_results_dir)
        logger.info("Nettoyage terminé avec succès.")
    except OSError as e:
        logger.error(f"Erreur lors de la suppression du dossier {mlm_results_dir}: {e}")
        # Alternative plus forcée si la première échoue
        !rm -rf {mlm_results_dir}
else:
    logger.info("Le dossier des résultats MLM n'existe pas, pas de nettoyage nécessaire.")

In [ ]:
# --- Étape 2.1 : Préparation et Tokenization des Données de Classification ---
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from datasets import Dataset
from transformers import AutoTokenizer

logger.info("Phase 2 : Préparation des données pour la Classification.")

# --- Mappage des Labels ---
df['vote_final'] = df['vote_final'].astype(str)
labels_list = sorted(df['vote_final'].unique())
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}
df['labels'] = df['vote_final'].map(label2id)
logger.info(f"Mappage des labels créé : {label2id}")

# --- Division en Train / Validation / Test ---
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['labels'])
logger.info(f"Taille des ensembles - Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

# --- Rééquilibrage par Sursampling (Oversampling) ---
logger.info("Rééquilibrage de l'ensemble d'entraînement...")
try:
    if len(train_df['labels'].value_counts()) > 1:
        majority_class_id = train_df['labels'].value_counts().idxmax()
        df_majority = train_df[train_df.labels == majority_class_id]
        df_minority = train_df[train_df.labels != majority_class_id]
        df_minority_oversampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)
        train_df_balanced = pd.concat([df_majority, df_minority_oversampled])
        logger.info(f"Distribution après rééquilibrage:\n{train_df_balanced['vote_final'].value_counts()}")
    else:
        logger.warning("Une seule classe détectée dans le jeu d'entraînement, pas de rééquilibrage effectué.")
        train_df_balanced = train_df
except Exception as e:
    logger.error(f"Erreur pendant le rééquilibrage : {e}. Utilisation du jeu de données original.")
    train_df_balanced = train_df

# --- Création des Datasets Hugging Face ---
train_dataset = Dataset.from_pandas(train_df_balanced)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# --- Tokenization ---
logger.info(f"Chargement du tokenizer adapté depuis {ADAPTED_LM_DIR}...")
classifier_tokenizer = AutoTokenizer.from_pretrained(ADAPTED_LM_DIR)

def tokenize_function_classifier(examples):
    return classifier_tokenizer(examples['message'], truncation=True, padding='max_length', max_length=128)

columns_to_remove = ['message', 'vote_final'] # On garde 'labels' !
train_tokenized = train_dataset.map(tokenize_function_classifier, batched=True, remove_columns=columns_to_remove)
val_tokenized = val_dataset.map(tokenize_function_classifier, batched=True, remove_columns=columns_to_remove)
test_tokenized = test_dataset.map(tokenize_function_classifier, batched=True, remove_columns=columns_to_remove)

logger.info("Préparation des données de classification terminée.")

### Étape 2.2 : Entraînement du Classifieur

Nous chargeons le modèle que nous avons adapté en Phase 1. Il connaît déjà notre vocabulaire. Nous le chargeons maintenant avec une tête de classification (`AutoModelForSequenceClassification`) et nous le fine-tunons sur nos données étiquetées.

In [ ]:
# --- Étape 2.2 : Préparation du Modèle de Classification (Congélation) ---
from transformers import AutoModelForSequenceClassification

logger.info(f"Chargement du modèle adapté depuis : {ADAPTED_LM_DIR}")
classifier_model = AutoModelForSequenceClassification.from_pretrained(
    ADAPTED_LM_DIR,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# --- CORRECTION PRINCIPALE : Utiliser 'roberta' au lieu de 'distilbert' ---
# Le modèle de base pour CamemBERT s'appelle 'roberta' en interne.
logger.info("Congélation du corps du Transformer pour le fine-tuning initial...")

# On gèle toutes les couches du corps du modèle
for param in classifier_model.roberta.parameters():
    param.requires_grad = False

# MAIS, on s'assure que la couche d'embedding reste entraînable pour qu'elle continue de s'adapter.
# Les embeddings font partie de 'roberta', donc on accède via `roberta.embeddings`
for param in classifier_model.roberta.embeddings.parameters():
    param.requires_grad = True

# La tête de classification (classifier_model.classifier) est automatiquement
# entraînable car elle est nouvelle et ses paramètres ont `requires_grad=True` par défaut.

# --- Vérification (inchangée) ---
trainable_params = sum(p.numel() for p in classifier_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in classifier_model.parameters())
logger.info(f"Paramètres entraînables (phase 1) : {trainable_params} / {total_params} ({100 * trainable_params / total_params:.2f}%)")
# Le log devrait maintenant s'afficher correctement, montrant que seul un petit pourcentage
# des poids est prêt à être entraîné.

In [ ]:
# Le code ci-dessous est CORRECT et fonctionnera après la mise à jour de l'environnement.

from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# ASSUREZ-VOUS QUE `label2id` EST DISPONIBLE ET CONTIENT BIEN LE MAPPING
hateful_class_id = label2id.get('hateful')
if hateful_class_id is None:
    hateful_class_id = 1
    logger.warning(f"Label 'hateful' non trouvé dans label2id. Utilisation de l'ID {hateful_class_id} comme classe positive par défaut.")

# Fonction de calcul des métriques, maintenant centrée sur le rappel
def compute_metrics_cls(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    recall_hateful = recall_score(labels, predictions, pos_label=hateful_class_id, zero_division=0)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average='weighted', zero_division=0),
        "recall_hateful": recall_hateful
    }

# Arguments d'entraînement avec Early Stopping basé sur le RAPPEL
training_args_phase1 = TrainingArguments(
    output_dir="/kaggle/working/classifier_results_phase1",
    num_train_epochs=15,
    learning_rate=5e-4,
    per_device_train_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    fp16=True,
    
    # Ces arguments sont maintenant reconnus par la nouvelle version de la bibliothèque
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    
    load_best_model_at_end=True, 
    metric_for_best_model="recall_hateful",
    greater_is_better=True,
    
    report_to="none"
)

# Création du Trainer
trainer_phase1 = Trainer(
    model=classifier_model,
    args=training_args_phase1,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=classifier_tokenizer,
    compute_metrics=compute_metrics_cls,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

logger.info("Début de la phase 1 de fine-tuning (optimisation du rappel)...")
trainer_phase1.train()

logger.info("Phase 1 de fine-tuning (rappel) terminée.")

In [ ]:
# --- Étape 2.4 : Décongélation et Fine-Tuning Final (Modèle Complet, optimisé pour le Rappel) ---
logger.info("Décongélation de toutes les couches pour le fine-tuning final...")
for param in classifier_model.parameters():
    param.requires_grad = True
logger.info("Tous les paramètres sont maintenant entraînables.")

# Arguments pour le fine-tuning final avec un learning rate très bas
training_args_final = TrainingArguments(
    output_dir="/kaggle/working/classifier_results_final",
    num_train_epochs=10, # Limite maximale d'époques
    
    # Taux d'apprentissage très bas pour un affinage de précision
    learning_rate=2e-5, 
    
    per_device_train_batch_size=BATCH_SIZE, # Assurez-vous que BATCH_SIZE est défini
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    fp16=True,
    
    # Mêmes stratégies que précédemment
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    
    # --- CHANGEMENT PRINCIPAL ICI ---
    load_best_model_at_end=True,
    metric_for_best_model="recall_hateful", # On continue de surveiller le rappel
    greater_is_better=True
)

# On crée un nouveau Trainer pour cette phase finale.
# Il utilise la même fonction `compute_metrics_cls` que précédemment.
trainer_final = Trainer(
    model=classifier_model, # Le modèle a maintenant toutes ses couches dégelées
    args=training_args_final,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=classifier_tokenizer,
    compute_metrics=compute_metrics_cls,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Patience plus courte pour la phase d'affinage
)

logger.info("Début du fine-tuning final (modèle complet, optimisation du rappel)...")
trainer_final.train()

# --- SAUVEGARDE DU MODÈLE FINAL ---
# `trainer_final.model` contient maintenant la meilleure version du modèle complet,
# basée sur le RAPPEL de la classe haineuse.
logger.info(f"Sauvegarde du meilleur modèle final (optimisé pour le rappel) dans : {FINAL_CLASSIFIER_DIR}")
trainer_final.save_model(FINAL_CLASSIFIER_DIR)
classifier_tokenizer.save_pretrained(FINAL_CLASSIFIER_DIR)

logger.info("Phase 2 (Fine-tuning de classification) terminée avec succès.")

### Étape 2.3 : Évaluation, Visualisation et Sauvegarde du Modèle Final

L'entraînement est terminé. Il est temps d'analyser en détail les performances de notre modèle fine-tuné. Nous allons :
1.  Sauvegarder le meilleur modèle et son tokenizer.
2.  Visualiser les courbes d'apprentissage pour confirmer que l'entraînement s'est bien déroulé.
3.  Calculer les métriques de performance (précision, rappel, F1-score).
4.  Afficher une matrice de confusion pour comprendre les types d'erreurs que le modèle commet.

In [ ]:
# --- Étape 2.5 : Évaluation Finale et Visualisation ---
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

logger.info("Évaluation finale sur le jeu de test (données jamais vues)...")
predictions_output = trainer_final.predict(test_tokenized)
y_preds = predictions_output.predictions.argmax(axis=1)
y_true = test_tokenized['labels']

# Rapport de classification (Précision, Rappel, F1-Score)
print("\n" + "="*50)
print("--- Rapport de Classification Final ---")
target_names_ordered = [id2label[i] for i in sorted(id2label.keys())]
print(classification_report(y_true, y_preds, target_names=target_names_ordered))
print("="*50 + "\n")

# Matrice de confusion
cm = confusion_matrix(y_true, y_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_ordered, yticklabels=target_names_ordered)
plt.title("Matrice de Confusion du Modèle Final sur le Jeu de Test")
plt.xlabel("Étiquette Prédite")
plt.ylabel("Étiquette Vraie")
plt.show()

# --- Visualisation des courbes d'entraînement de la dernière phase ---
log_history_cls = trainer_final.state.log_history
df_log_cls = pd.DataFrame(log_history_cls)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))
fig.suptitle("Performances du Fine-Tuning Final (Phase 2.4)", fontsize=16)

# Filtrer les données pour les graphiques
train_loss_cls = df_log_cls[df_log_cls['loss'].notna()]
eval_loss_cls = df_log_cls[df_log_cls['eval_loss'].notna()]
eval_acc_cls = df_log_cls[df_log_cls['eval_f1'].notna()] # On affiche le F1-score

# Graphique de la Perte
if not eval_loss_cls.empty:
    ax1.plot(train_loss_cls['epoch'], train_loss_cls['loss'], marker='o', label="Training Loss")
    ax1.plot(eval_loss_cls['epoch'], eval_loss_cls['eval_loss'], marker='o', label="Validation Loss")
    ax1.set_title("Perte (Loss)")
    ax1.set_xlabel("Époque"); ax1.set_ylabel("Perte"); ax1.grid(True); ax1.legend()

# Graphique du F1-score
if not eval_acc_cls.empty:
    ax2.plot(eval_acc_cls['epoch'], eval_acc_cls['eval_f1'], marker='o', color='purple', label="Validation F1-Score")
    ax2.set_title("F1-Score")
    ax2.set_xlabel("Époque"); ax2.set_ylabel("F1-Score"); ax2.grid(True); ax2.legend()

plt.show()

### Étape 2.4 : Test sur des Exemples Concrets avec une Pipeline

La méthode la plus simple pour utiliser notre modèle est de le charger dans une `pipeline` Hugging Face. Cela s'occupe de toute la pré-traitement et du post-traitement pour nous.

In [ ]:
from transformers import pipeline

# Charger le modèle final dans une pipeline pour un test facile
logger.info("Chargement de la pipeline avec le modèle final...")
# S'assurer d'utiliser le GPU s'il est disponible
device = 0 if torch.cuda.is_available() else -1
final_pipe = pipeline("text-classification", model=FINAL_CLASSIFIER_DIR, device=device)

# Exemples de test
test_phrases = [
    "Je te déteste, tu n'es qu'un idiot.",
    "Passe une excellente journée, merci pour ton aide !",
    "Ce nkwada pense qu'il peut nous tromper.", # Exemple avec vocabulaire spécifique
    "C'est une honte pour notre pays.",
    "Arrête de dire des bêtises, tu es nul",
    "tu es deguelasse",
    "c'est pitoyable",
]

logger.info("\n--- Test du modèle final sur des exemples concrets ---")
for phrase in test_phrases:
    result = final_pipe(phrase)
    print(f"Phrase: '{phrase}'\nPrédiction: {result}\n")

In [ ]:
# S'assurer que la pipeline est bien chargée
try:
    final_pipe
except NameError:
    from transformers import pipeline
    logger.info("Rechargement de la pipeline pour l'interface interactive...")
    device = 0 if torch.cuda.is_available() else -1
    final_pipe = pipeline("text-classification", model=FINAL_CLASSIFIER_DIR, device=device)

def classify_interactive():
    """Lance une boucle de classification interactive."""
    print("\n" + "="*60)
    print("      Interface de Classification de Discours Haineux")
    print("="*60)
    print("Entrez une phrase à analyser. Tapez 'quitter' pour arrêter.")
    print("-" * 60)

    while True:
        # Demander une phrase à l'utilisateur
        user_input = input("Votre phrase > ")

        # Condition de sortie
        if user_input.lower() == 'quitter':
            print("Au revoir !")
            break
        
        # Vérifier que l'input n'est pas vide
        if not user_input.strip():
            print("Veuillez entrer une phrase non vide.")
            continue

        # Faire la prédiction avec la pipeline
        result = final_pipe(user_input)[0] # On prend le premier élément de la liste
        label = result['label']
        score = result['score']

        # Afficher le résultat de manière lisible
        print(f"  -> Prédiction : '{label}' (Confiance : {score:.2%})")
        print("-" * 60)

# Lancer l'interface interactive
classify_interactive()